In [1]:
import numpy as np
import scipy.integrate as integrate
from numpy.linalg import inv
import math as m
from functions import *
import importlib

%load_ext autoreload

%autoreload 2

np.set_printoptions(suppress=True)

This next block will create a class for all of our inputs to be placed into. This will allow for all inputs to have easily accessed names, values, and units.

In [2]:
class Input:
    def __init__(self, name, value, unit):
        self.name = name
        self.value = value
        #self.valueType = type(value)
        self.unit = unit
    
    def printValues(self):
        for attr, value in self.__dict__.items():
            print("\nAttribute: " + str(attr))
            print("Value: {0}".format(value))

# Please fill in all inputs and their units below
# v v v v v v v v v v v v v v v v v v v v v v v v v v v v v v v v v v v v v v v 

In [3]:
YoungsModulus = 30*(10**3)
YM_Unit = "ksi"

Area = 1
A_Unit = "in2"

P1y = -2.5
Force_Unit = "kip"

# node connections for each element
nArray = np.array([[1,2], [1,3], [2,3], [2,4], [3,4], [3,5], [4,5]])

# list of lengths corresponding to each element
# the length of element n is contained in lengths
lArray = np.array([6, 3*(m.sqrt(3)), 3, 6, 3*(m.sqrt(3)), 6, 3])
lArray *= 12

Length_unit = "in"

# list of angles relative to positive x axis for each element
aArray = np.array([(7/6), 1, 0.5, (5/6), 1, (5/6), 0.5])
aArray *= m.pi

Angle_unit = "rad"

# input boundary conditions below
# for displacements: [node#, u, v]
# for forces: [node#, Fx, Fy]

uv_BCs = np.array([[4,0,0],[5,0,0]])

force_BCs = np.array([[1, 0, P1y],[4, None, None], [5, None, None]])

# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [4]:
E = Input("Young's Modulus", 30*(10**3), "ksi")

A = Input("Area", 1, "in^2")

nodes = Input("Node numbers", nArray, "None")
nodes.printValues()

lengths = Input("Lengths", lArray, Length_unit)
lengths.printValues()

angles = Input("Element Angles (relative to the positive x axis)", aArray, Angle_unit)
angles.printValues()


Attribute: name
Value: Node numbers

Attribute: value
Value: [[1 2]
 [1 3]
 [2 3]
 [2 4]
 [3 4]
 [3 5]
 [4 5]]

Attribute: unit
Value: None

Attribute: name
Value: Lengths

Attribute: value
Value: [72.         62.35382907 36.         72.         62.35382907 72.
 36.        ]

Attribute: unit
Value: in

Attribute: name
Value: Element Angles (relative to the positive x axis)

Attribute: value
Value: [3.66519143 3.14159265 1.57079633 2.61799388 3.14159265 2.61799388
 1.57079633]

Attribute: unit
Value: rad


number of elements is the number of rows in nodes
<br>number of nodes is the maximum number found in nodes

In [5]:
numElems = len(nodes.value)
print(numElems)
numNodes = nodes.value.max()
print(numNodes)

7
5


two reactions per node
<br>uvList -> list of unknowns (node displacements)
<br>we can input our known values -> fixed_nodes at nodes 4 & 5
<br>these fixed nodes numbers are taken straight from diagram (not adjusted for python counting)
<br><br>u and v for any given node is given as (node# - 1)* 2 and ((node# -1)* 2) + 1 respectively.


In [6]:
uvList = [None]*(numNodes*2)
print("displacement bcs: [node number, x disp (u), y disp (v)]")
print(uv_BCs)

adjust_array(uvList, uv_BCs, "displacements")
print("u & v list:")
print(uvList)

displacement bcs: [node number, x disp (u), y disp (v)]
[[4 0 0]
 [5 0 0]]
adjusting array to account for: displacements boundary conditions
case 1 for BC 1
Case 2 for BC 1
case 1 for BC 2
Case 2 for BC 2
done adjusting displacements array!
u & v list:
[None, None, None, None, None, None, 0, 0, 0, 0]


Same procedure as displacement BCs for forces
<br>Input BCs given as : [node #, x force, y force]

In [7]:
#forceList = [0]*(numNodes*2)
forceList = np.zeros(((numNodes*2), 1))

nodesWithLoads = []

loadedNodes = 0

for i in range(len(force_BCs)):
    if force_BCs[i,1] or force_BCs[i,2] != None:
        loadedNodes += 1
        nodesWithLoads.append((force_BCs[i,0] - 1))


print("number of nodes with concentrated loads: {0}".format(loadedNodes))
print("at nodes: {0}".format(nodesWithLoads))

#adjust force array given concentrated loading conditions
adjust_array(forceList, force_BCs, "forces")

print("force list: ")
print(forceList)

number of nodes with concentrated loads: 1
at nodes: [0]
adjusting array to account for: forces boundary conditions
case 3 for BC 1
Case 2 for BC 1
case 1 for BC 2
Case 2 for BC 2
case 1 for BC 3
Case 2 for BC 3
done adjusting forces array!
force list: 
[[ 0. ]
 [-2.5]
 [ 0. ]
 [ 0. ]
 [ 0. ]
 [ 0. ]
 [ nan]
 [ nan]
 [ nan]
 [ nan]]


Global [k] matrix is a symmetric array of shape (numNodes x 2, numNodes x 2)
<br>Bar Constants are one per element, stored in a numElemes * 1 array.

In [8]:
globalK = np.zeros(((numNodes*2),(numNodes*2)))
print(globalK.shape)

barConstant = np.zeros((numElems,1))
for i in range(numElems):
    barConstant[i] = (E.value*A.value)/lengths.value[i]

print("Bar Constants = ")
print(barConstant)

(10, 10)
Bar Constants = 
[[416.66666667]
 [481.12522432]
 [833.33333333]
 [416.66666667]
 [481.12522432]
 [416.66666667]
 [833.33333333]]


## Creating the finite element equation for each Truss Element
### Combining into the global K array

In [9]:
for i in range(numElems):
    kMatrix = create_truss_k(angles.value[i])
    kMatrix *= barConstant[i]
    print("\nk matrix for element {1}: \n{0}".format(kMatrix, (i+1)))
    #print(kMatrix)
    startNode = nodes.value[i,0]
    endNode = nodes.value[i,1]
    print("Element " + str(i+1) + " starting at node: " + str(startNode) + " and ending at node: " + str(endNode))
    #print(np.vsplit(kMatrix, 2))
    Sn = (startNode - 1)*2
    En = (endNode - 1)*2
    #first, adjust 0,0 -> then, 2,0; 0,2; and 2,2
    block_array_adjust(Sn, Sn, globalK, 0, 0, kMatrix)
    block_array_adjust(En, Sn, globalK, 2, 0, kMatrix)
    block_array_adjust(Sn, En, globalK, 0, 2, kMatrix)
    block_array_adjust(En, En, globalK, 2, 2, kMatrix)

print("\nGlobal K matrix: \n{0}".format(globalK))
unmodified_globalK = globalK.copy()
check_Symmetric(globalK)


k matrix for element 1: 
[[ 312.5         180.42195912 -312.5        -180.42195912]
 [ 180.42195912  104.16666667 -180.42195912 -104.16666667]
 [-312.5        -180.42195912  312.5         180.42195912]
 [-180.42195912 -104.16666667  180.42195912  104.16666667]]
Element 1 starting at node: 1 and ending at node: 2

k matrix for element 2: 
[[ 481.12522432   -0.         -481.12522432    0.        ]
 [  -0.            0.            0.           -0.        ]
 [-481.12522432    0.          481.12522432   -0.        ]
 [   0.           -0.           -0.            0.        ]]
Element 2 starting at node: 1 and ending at node: 3

k matrix for element 3: 
[[   0.            0.           -0.           -0.        ]
 [   0.          833.33333333   -0.         -833.33333333]
 [  -0.           -0.            0.            0.        ]
 [  -0.         -833.33333333    0.          833.33333333]]
Element 3 starting at node: 2 and ending at node: 3

k matrix for element 4: 
[[ 312.5        -180.42195912

In [10]:
#print(inv(globalK))
u = np.linalg.inv(globalK).dot(forceList)
print(u)

[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]


In [11]:
j = 0
while j < numNodes*2:
    print(str(j))
    if j in nodesWithLoads:
        j+=2
    #first case: displacement BC = 0. In this case, set combined force values to 0, and alter 
    #globalK to reflect the changes.
    elif uvList[j] == 0:
        print("u = 0 at row: " + str(j))
        forceList[j] = 0 
        globalK[j] = 0
        globalK[j,j] = 1
        j+=1
    elif uvList[j] == None:
        print("none detected at row " + str(j))
        j+=1
    else: #when we have a known, non-zero displacement boundary condition
        #alter global stiffness to all zeros and one 1, and set force equal to displacement BC
        print("row " + str(j) + " altered")
        print(uvList[j])
        forceList[j] = uvList[j]
        for c in range(numNodes):
            if (globalK[j,c] > 0) == True:
                globalK[j,c] = 1
            else:
                globalK[j,c] = 0
        j+=1

                
print(globalK)
print(forceList)

0
2
none detected at row 2
3
none detected at row 3
4
none detected at row 4
5
none detected at row 5
6
u = 0 at row: 6
7
u = 0 at row: 7
8
u = 0 at row: 8
9
u = 0 at row: 9
[[ 793.62522432  180.42195912 -312.5        -180.42195912 -481.12522432
     0.            0.            0.            0.            0.        ]
 [ 180.42195912  104.16666667 -180.42195912 -104.16666667    0.
    -0.            0.            0.            0.            0.        ]
 [-312.5        -180.42195912  625.            0.           -0.
    -0.         -312.5         180.42195912    0.            0.        ]
 [-180.42195912 -104.16666667    0.         1041.66666667   -0.
  -833.33333333  180.42195912 -104.16666667    0.            0.        ]
 [-481.12522432    0.           -0.           -0.         1274.75044865
  -180.42195912 -481.12522432    0.         -312.5         180.42195912]
 [   0.           -0.           -0.         -833.33333333 -180.42195912
   937.5           0.           -0.          180.4219

In [12]:
#globalK.round(0)
globalK_inv = inv(globalK)
#print(globalK_inv)

node = 1

uFinal = np.matmul(globalK_inv, forceList)

for i in range(len(uFinal)):
    if check_even(i) == True:
        print("u{0} = {1}".format((node), uFinal[i]))
    else:
        print("v{0} = {1}".format((node), uFinal[i]))
        node += 1 

u1 = [0.]
v1 = [-0.18717691]
u2 = [-0.05403332]
v2 = [-0.06958846]
u3 = [-0.009]
v3 = [-0.06358846]
u4 = [0.]
v4 = [0.]
u5 = [0.]
v5 = [0.]


Forces can be found from [K]*[u]

In [13]:
forcesFinal = np.matmul(unmodified_globalK, uFinal)
for i in range(len(forceList)):
    if forceList[i] != forcesFinal[i] and forceList[i] != 0 and forceList[i] != None:
        forcesFinal[i] = forceList[i]
print(forcesFinal)

node = 1

for i in range(len(forcesFinal)):
    if check_even(i) == True:
        print("fx{0} = {1}".format((node), forcesFinal[i]))
    else:
        print("fy{0} = {1}".format((node), forcesFinal[i]))
        node += 1 


[[-0.        ]
 [-2.5       ]
 [ 0.        ]
 [ 0.        ]
 [-0.        ]
 [-0.        ]
 [ 8.66025404]
 [-2.5       ]
 [-8.66025404]
 [ 5.        ]]
fx1 = [-0.]
fy1 = [-2.5]
fx2 = [0.]
fy2 = [0.]
fx3 = [-0.]
fy3 = [-0.]
fx4 = [8.66025404]
fy4 = [-2.5]
fx5 = [-8.66025404]
fy5 = [5.]


## Finding Stresses

In [14]:
stressFinal = np.zeros((numElems, 1))
print(stressFinal)
for i in range(numElems):
    startNode = nodes.value[i,0]
    endNode = nodes.value[i,1]
    print("Element {0} starts at node: {1} and ends at node: {2}".format((i+1), startNode, endNode))
    #print(np.vsplit(kMatrix, 2))
    Sn = (startNode - 1)*2
    En = (endNode - 1)*2
    print("adjusted start node: {0} adjusted end node: {1}".format(Sn, En))
    trigMatrix = create_truss_stress_trig_matrix(angles.value[i])
    print("trigMatrix = \n{0}".format(trigMatrix))
    local_uMatrix = np.array([
        uFinal[Sn], 
        uFinal[(Sn + 1)], 
        uFinal[En], 
        uFinal[(En+1)]
        ])
    print("local u matrix: \n{0}".format(local_uMatrix))
    localLength = np.array([[(-1/lengths.value[i]), 1/lengths.value[i]]])
    print("local length matrix: \n{0}".format(localLength))

    indStress = (trigMatrix @ local_uMatrix)
    indStress = localLength @ indStress
    indStress *= E.value

    stressFinal[i] = indStress
    
    
    print("Stress in element = {0}".format(indStress))

    

[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
Element 1 starts at node: 1 and ends at node: 2
adjusted start node: 0 adjusted end node: 2
trigMatrix = 
[[-0.8660254 -0.5        0.         0.       ]
 [ 0.         0.        -0.8660254 -0.5      ]]
local u matrix: 
[[ 0.        ]
 [-0.18717691]
 [-0.05403332]
 [-0.06958846]]
local length matrix: 
[[-0.01388889  0.01388889]]
Stress in element = [[-5.]]
Element 2 starts at node: 1 and ends at node: 3
adjusted start node: 0 adjusted end node: 4
trigMatrix = 
[[-1.  0.  0.  0.]
 [ 0.  0. -1.  0.]]
local u matrix: 
[[ 0.        ]
 [-0.18717691]
 [-0.009     ]
 [-0.06358846]]
local length matrix: 
[[-0.01603751  0.01603751]]
Stress in element = [[4.33012702]]
Element 3 starts at node: 2 and ends at node: 3
adjusted start node: 2 adjusted end node: 4
trigMatrix = 
[[0. 1. 0. 0.]
 [0. 0. 0. 1.]]
local u matrix: 
[[-0.05403332]
 [-0.06958846]
 [-0.009     ]
 [-0.06358846]]
local length matrix: 
[[-0.02777778  0.02777778]]
Stress in element = [[5.]]


In [15]:
print(stressFinal)

[[-5.        ]
 [ 4.33012702]
 [ 5.        ]
 [-5.        ]
 [-4.33012702]
 [10.        ]
 [ 0.        ]]


In [16]:
np.set_printoptions(suppress=True, formatter={'float':"{:0.4f}".format})
printNodalDisplacements(uFinal, lengths.unit)
printStresses(stressFinal,E.unit)

████████████████████████████████████████████████████████████████████████████████████████████████████
The nodal displacements, in [in], ordered from lowest to highest numbered node are:
[[0.0000]
 [-0.1872]
 [-0.0540]
 [-0.0696]
 [-0.0090]
 [-0.0636]
 [0.0000]
 [0.0000]
 [0.0000]
 [0.0000]]
████████████████████████████████████████████████████████████████████████████████████████████████████
████████████████████████████████████████████████████████████████████████████████████████████████████

The element axial stresses, in [ksi], ordered from lowest to highest numbered element are:
[[-5.0000]
 [4.3301]
 [5.0000]
 [-5.0000]
 [-4.3301]
 [10.0000]
 [0.0000]]

Note: a (+) value indicates the element is in tension, while a (-) value in compression.

████████████████████████████████████████████████████████████████████████████████████████████████████
